In [0]:
# Load the combined_mine_data table
combined_mine_data_df = spark.table("default.combined_mine_data")

# Example query: Select specific columns and filter by a condition
query = """
SELECT distinct accident_dt, latitude, longitude,current_mine_name, mine_id
FROM default.combined_mine_data
where fiscal_yr > 2020
"""

# Execute the query
result_df = spark.sql(query)

# Display the result
display(result_df)

In [0]:
print(result_df.count())


In [0]:
%pip install pandas tqdm requests
%pip install meteostat geopy pandas tqdm


In [0]:
 %restart_python

In [0]:
# Install dependencies if needed:
# pip install meteostat geopy pandas tqdm numpy scipy

from meteostat import Stations, Daily
from scipy.spatial import cKDTree
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --------------------------
# CONFIGURATION
# --------------------------
OUTPUT_FILE = 'mine_accident_weather.csv'
MISSING_LOG_FILE = 'missing_weather_log.csv'
FALLBACK_NEAREST = 3  # Try up to 3 nearest stations
MAX_WORKERS = 10      # Number of parallel API calls

# --------------------------
# STEP 1: PREPARE DATA
# --------------------------
# # result_df must include: current_mine_name, latitude, longitude, accident_dt
# #result_df = pd.DataFrame()  
# # Initialize result_df before use 
# required_cols = {'current_mine_name', 'latitude', 'longitude', 'accident_dt'}

# if not required_cols.issubset(result_df.columns):
#     raise ValueError(f"result_df must have columns: {required_cols}")

result_df = result_df.toPandas()
# print(result_df)  # Ensure result_df is loaded or initialized before this line
# print(f"✅ Total accident rows: {len(result_df)}")

# Clean data types
result_df['latitude'] = pd.to_numeric(result_df['latitude'], errors='coerce')
result_df['longitude'] = pd.to_numeric(result_df['longitude'], errors='coerce')
result_df['accident_dt'] = pd.to_datetime(result_df['accident_dt'], errors='coerce')

# Drop rows with invalid coordinates or dates
result_df = result_df.dropna(subset=['latitude', 'longitude', 'accident_dt']).reset_index(drop=True)

# --------------------------
# STEP 2: NEAREST WEATHER STATION
# --------------------------
print("📡 Fetching weather stations...")
stations = Stations().fetch().reset_index()
stations['latitude'] = pd.to_numeric(stations['latitude'], errors='coerce')
stations['longitude'] = pd.to_numeric(stations['longitude'], errors='coerce')
stations = stations.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)

print("🏎️ Finding nearest stations for all accidents...")
station_coords = stations[['latitude', 'longitude']].to_numpy()
mine_coords = result_df[['latitude', 'longitude']].to_numpy()

# Build KDTree for fast lookup
tree = cKDTree(station_coords)
distances, indices = tree.query(mine_coords, k=FALLBACK_NEAREST)

# Add nearest station info to result_df
result_df['station_id'] = stations.iloc[indices[:, 0]]['id'].values
result_df['station_name'] = stations.iloc[indices[:, 0]]['name'].values
result_df['distance_km'] = distances[:, 0]

# --------------------------
# STEP 3: FETCH WEATHER DATA (BATCHED)
# --------------------------
print("🌤️ Fetching weather data in parallel...")

# Group by station to avoid redundant API calls
station_groups = result_df.groupby('station_id')['accident_dt'].apply(lambda x: sorted(set(x))).to_dict()

def fetch_weather_for_station(station_id, dates):
    """
    Fetch weather data for a station and filter for accident dates
    """
    try:
        start_date = min(dates)
        end_date = max(dates)
        data = Daily(station_id, start_date, end_date).fetch().reset_index()
        if data.empty:
            return station_id, None
        else:
            # Filter only accident dates
            filtered = data[data['time'].isin(dates)].copy()
            filtered['station_id'] = station_id
            return station_id, filtered
    except Exception as e:
        print(f"❌ Error fetching data for station {station_id}: {e}")
        return station_id, None

# Parallelize API calls
station_weather = {}
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(fetch_weather_for_station, sid, dates): sid for sid, dates in station_groups.items()}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading weather"):
        sid, weather = future.result()
        station_weather[sid] = weather

# --------------------------
# STEP 4: MERGE WEATHER DATA
# --------------------------
print("🔄 Merging weather data with accidents...")
weather_df = pd.concat([df for df in station_weather.values() if df is not None], ignore_index=True)

# Merge weather details into result_df
final_df = result_df.merge(
    weather_df,
    how='left',
    left_on=['station_id', 'accident_dt'],
    right_on=['station_id', 'time']
).drop(columns=['time'])

print(final_df)


In [0]:
# Convert columns with null values to 0
final_df = final_df.fillna(0)
final_df.display()

In [0]:
# --------------------------
# STEP 5: CLASSIFY WEATHER
# --------------------------
def classify_weather(row):
    """Classify weather as Good, Moderate, or Bad"""
    if pd.isna(row['tavg']):
        return "Unknown Weather"
    if 10 <= row['tavg'] <= 30 and row['prcp'] < 20:  # Adjust thresholds if needed
        return "Good Weather"
    elif row['tavg'] < -10 or row['tavg'] > 40 or row['prcp'] > 50:
        return "Bad Weather"
    else:
        return "Moderate Weather"

final_df['weather_summary'] = final_df.apply(classify_weather, axis=1)

# --------------------------
# STEP 6: SAVE OUTPUT
# --------------------------
# final_df.to_csv(OUTPUT_FILE, index=False)
# print(f"✅ Accident weather data saved to {OUTPUT_FILE}")

# # Log missing weather
# missing_weather = final_df[final_df['tavg'].isna()][['current_mine_name', 'accident_dt']]
# if not missing_weather.empty:
#     missing_weather.to_csv(MISSING_LOG_FILE, index=False)
#     print(f"⚠️ Logged {len(missing_weather)} accident rows with missing weather to '{MISSING_LOG_FILE}'.")

# Preview
print(final_df.head())


In [0]:
# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(final_df)
# Drop the existing table if it exists
spark.sql("DROP TABLE IF EXISTS default.mine_accident_weather")

# Save spark_df as a Delta table
spark_df.write.mode("overwrite").saveAsTable("default.mine_accident_weather")

In [0]:
print(final_df['tavg'].notna().sum(), "rows have weather data")
print(final_df['tavg'].isna().sum(), "rows missing weather data")
